# Load, Fuel Mix, and LMP Data

This notebook walk through how to use `gridstatus` to access to the data availabe on [OASIS](http://oasis.caiso.com/).

While we will be using CAISO in this example, most of this API will also work with all other ISOs.

In [1]:
import gridstatus
import pandas as pd
import plotly.express as px

In [2]:
caiso = gridstatus.CAISO()

## Historical Fuel Mix

In [3]:
start = pd.Timestamp("April 18 2018").normalize()
end = pd.Timestamp.now().normalize()
mix_df = caiso.get_fuel_mix(start, end=end, verbose=False)

100%|██████████| 2727/2727 [10:02<00:00,  4.52it/s]


In [10]:
pip install kaleido

Note: you may need to restart the kernel to use updated packages.


In [ ]:
mix_df['Time'] = pd.to_datetime(mix_df['Time'])
hourly_mix = mix_df.set_index('Time').resample('h').mean()
monthly_mix = (
    hourly_mix.select_dtypes(include='number')
    .resample('MS')
    .sum()
    .reset_index()
    .iloc[1:-1]
)

top_sources = (
    monthly_mix[monthly_mix.columns[1:]]
    .sum()
    .sort_values(ascending=False)
    .index.tolist()
)

fig = px.bar(monthly_mix, x="Time", y=top_sources, title="Fuel Mix by Month - CAISO")
fig.show()  # Use default renderer for Jupyter

## Historical Load

In [14]:
start = pd.Timestamp("April 18 2018").normalize()
end = pd.Timestamp.now().normalize()
load_df = caiso.get_load(start, end=end)

  0%|          | 0/2727 [00:00<?, ?it/s]2025-10-05 17:05:09 - INFO - Fetching URL: https://www.caiso.com/outlook/history/20180418/demand.csv?_=1759709109
INFO	Task(Task-2) gridstatus:caiso.py:_get_historical()- Fetching URL: https://www.caiso.com/outlook/history/20180418/demand.csv?_=1759709109
  0%|          | 1/2727 [00:00<06:57,  6.52it/s]2025-10-05 17:05:09 - INFO - Fetching URL: https://www.caiso.com/outlook/history/20180419/demand.csv?_=1759709109
INFO	Task(Task-2) gridstatus:caiso.py:_get_historical()- Fetching URL: https://www.caiso.com/outlook/history/20180419/demand.csv?_=1759709109
  0%|          | 2/2727 [00:00<07:00,  6.48it/s]2025-10-05 17:05:09 - INFO - Fetching URL: https://www.caiso.com/outlook/history/20180420/demand.csv?_=1759709109
INFO	Task(Task-2) gridstatus:caiso.py:_get_historical()- Fetching URL: https://www.caiso.com/outlook/history/20180420/demand.csv?_=1759709109
  0%|          | 3/2727 [00:00<07:57,  5.71it/s]2025-10-05 17:05:09 - INFO - Fetching URL: https

Error: 2020-03-08 02:00:00
Args: {'self': <gridstatus.caiso.CAISO object at 0x0000019DDAE48D70>, 'date': Timestamp('2020-03-08 00:00:00-0800', tz='US/Pacific')}



2025-10-05 17:07:12 - INFO - Fetching URL: https://www.caiso.com/outlook/history/20200310/demand.csv?_=1759709232
INFO	Task(Task-2) gridstatus:caiso.py:_get_historical()- Fetching URL: https://www.caiso.com/outlook/history/20200310/demand.csv?_=1759709232
 25%|██▌       | 693/2727 [02:03<05:35,  6.05it/s]2025-10-05 17:07:12 - INFO - Fetching URL: https://www.caiso.com/outlook/history/20200311/demand.csv?_=1759709232
INFO	Task(Task-2) gridstatus:caiso.py:_get_historical()- Fetching URL: https://www.caiso.com/outlook/history/20200311/demand.csv?_=1759709232
 25%|██▌       | 694/2727 [02:03<05:59,  5.66it/s]2025-10-05 17:07:13 - INFO - Fetching URL: https://www.caiso.com/outlook/history/20200312/demand.csv?_=1759709233
INFO	Task(Task-2) gridstatus:caiso.py:_get_historical()- Fetching URL: https://www.caiso.com/outlook/history/20200312/demand.csv?_=1759709233
 25%|██▌       | 695/2727 [02:04<05:56,  5.71it/s]2025-10-05 17:07:13 - INFO - Fetching URL: https://www.caiso.com/outlook/history/2

Errors that occurred while getting data:
[{'date': Timestamp('2020-03-08 00:00:00-0800', tz='US/Pacific'),
  'self': <gridstatus.caiso.CAISO object at 0x0000019DDAE48D70>}]


In [16]:
load_df['Time'] = pd.to_datetime(load_df['Time'])
hourly_load = load_df.set_index('Time').resample('h').mean()
monthly_load = (
    hourly_load.select_dtypes(include='number')
    .resample('MS')
    .sum()
    .reset_index()
    .iloc[1:-1]
)

top_sources = (
    monthly_load[monthly_load.columns[1:]]
    .sum()
    .sort_values(ascending=False)
    .index.tolist()
)

fig = px.bar(monthly_load, x="Time", y=top_sources, title="Historical Load by Month - CAISO")
fig.show()  # Use default renderer for Jupyter

## Historical Locational Marginal Pricing (LMP)

You can supply whatever nodes or market you'd like, but for now let's download data for 3 trading hubs in the Day Head Hourly Market.

Note: CAISO only provides last 39 months of data.

In [ ]:
start = pd.Timestamp("Jan 1 2020").normalize()
end = pd.Timestamp.now().normalize()

locations = ["TH_NP15_GEN-APND", "TH_SP15_GEN-APND", "TH_ZP26_GEN-APND"]

lmp_df = caiso.get_lmp(
    start=start, end=end, market="DAY_AHEAD_HOURLY", locations=locations, sleep=5
)

In [ ]:
negative_lmps = lmp_df[lmp_df["LMP"] < 0].set_index("Time")
negative_per_month = (
    negative_lmps.groupby("Location").resample("MS")["LMP"].count().reset_index()
)
fig = px.bar(
    negative_per_month,
    x="Time",
    y="LMP",
    title="Negative LMPs per Month - CAISO",
    color="Location",
)
fig.update_yaxes(title="Number of Negative LMPs")
fig.show("svg", width=1200, height=600)